In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import os

In [2]:
data = pd.read_csv('data/train/trainx.txt', sep='\t')
features = data.columns.values
print("特征如下：{} \n".format(features))
print("数据数目: ", len(data))

特征如下：['ITEM_NAME' 'TYPE'] 

数据数目:  499447


## 数据清洗
* 替换一些字符 类似于 （ ） ‘ ’ _

In [3]:
import re
from zhon.hanzi import punctuation
from string import punctuation as english_punc
def  rm_punc(strs):
    return re.sub(r"[{}]+".format(punctuation + english_punc)," ",strs)
data[features[0]]  = data[features[0]].apply(rm_punc)
data.head()

,ITEM_NAME,TYPE
0,腾讯QQ黄钻三个月QQ黄钻3个月季卡官方自动充值可查时间可续费,本地生活--游戏充值--QQ充值
1,腾讯q币充值30元qq币30qb30q币自动充值,本地生活--游戏充值--QQ充值
2,腾讯QQ红钻包月卡 QQ红钻一个月QQ红钻1个月,本地生活--游戏充值--QQ充值
3,腾讯QQ蓝钻贵族31个月直充,本地生活--游戏充值--QQ充值
4,腾讯QQ币148元148QQ币148个直充148Q币148个Q币148个QQB★自动充值,本地生活--游戏充值--QQ充值


In [4]:
# 因为有的label里本身就存在空格，所以先要把空格去掉，再去掉- - ， 再以空格划分
data = shuffle(data)
data_y = data[features[1]]

# 取训练数据和测试数据
train = data.sample(frac= 0.8).reset_index()
test = data.sample(frac= 0.2).reset_index()


print("训练数据数目:  ", len(train))
print("测试数据数目:  ", len(test))

train_x  = train[features[0]]
train_y  = train[features[1]]

test_x  = test[features[0]]
test_y  = test[features[1]]

print(train_x.shape, test_x.shape)

# 分出数据集 训练集 测试集 数据集用来训练词向量
data_y = data_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
data_y= [each.split() for each in data_y]

train_y = train_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
train_y = [each.split() for each in train_y]

test_y = test_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
test_y = [each.split() for each in test_y]

训练数据数目:   399558
测试数据数目:   99889
(399558,) (99889,)


## 分词

In [5]:
import jieba
from random import shuffle
def participle(data, doshuffle = False):
    words = []
    for i in range(len(data)):
        result=[]
        seg_list = jieba.cut(data.iloc[i])
        for w in seg_list :#读取每一行分词
            if w != " ":
                result.append(w)
        # 打乱每条数据的分词
        if doshuffle:
            shuffle(result)
        words.append(result)#将该行分词写入列表形式的总分词列表
    return words

In [6]:
# 每条数据的分词结果是个长度不等的词列表
word_data = participle(data[features[0]], doshuffle = False)
word_data_train = participle(train_x,doshuffle = False)
word_data_test = participle(test_x, doshuffle = False)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Richado\AppData\Local\Temp\jieba.cache
Loading model cost 1.674 seconds.
Prefix dict has been built succesfully.


In [7]:
data[features[0]][0]

'腾讯QQ黄钻三个月QQ黄钻3个月季卡官方自动充值可查时间可续费'

In [8]:
from random import shuffle
word_data[8]

['凌度',
 'BL950',
 '行车',
 '记录仪',
 '安霸',
 'A7L70',
 '方案',
 '高清',
 '广角',
 '170',
 '度',
 'HDR',
 '高',
 '动态',
 'BL950',
 '升级版',
 '标配',
 '64G',
 '卡']

## 标签分离(一个标签分成三个标签)

In [9]:
train_y = pd.DataFrame(train_y, columns=['label1', 'label2', 'label3'])
test_y = pd.DataFrame(test_y, columns=['label1', 'label2', 'label3'])
data_y = pd.DataFrame(data_y, columns=['label1', 'label2', 'label3'])

In [10]:
len(data_y["label1"].unique())

22

In [11]:
len(data_y["label2"].unique())

191

In [12]:
len(data_y["label3"].unique())

1192

## 标签onn_hot

In [13]:
from keras.utils import to_categorical
# 先生成唯一数组
y_label1 = data_y['label1'].unique().tolist()
y_label2 = data_y['label2'].unique().tolist()
y_label3 = data_y['label3'].unique().tolist()

# 获取在唯一数组中的索引(训练集和测试集各有3个标签需要处理)
train_y_label1_map = train_y['label1'].apply(lambda x: y_label1.index(x))
train_y_label2_map = train_y['label2'].apply(lambda x: y_label2.index(x))
train_y_label3_map = train_y['label3'].apply(lambda x: y_label3.index(x))
test_y_label1_map = test_y['label1'].apply(lambda x: y_label1.index(x))
test_y_label2_map = test_y['label2'].apply(lambda x: y_label2.index(x))
test_y_label3_map = test_y['label3'].apply(lambda x: y_label3.index(x))

Using TensorFlow backend.


In [14]:
# 生成对应one-hot(用做训练模型的标签)
train_y_label1_one_hot = to_categorical(train_y_label1_map, 22)
train_y_label2_one_hot = to_categorical(train_y_label2_map, 191)
train_y_label3_one_hot = to_categorical(train_y_label3_map, 1192)
test_y_label1_one_hot = to_categorical(test_y_label1_map, 22)
test_y_label2_one_hot = to_categorical(test_y_label2_map, 191)
test_y_label3_one_hot = to_categorical(test_y_label3_map, 1192)

In [15]:
from keras.preprocessing.text import *
from keras.preprocessing import sequence
tok = Tokenizer()
tok.fit_on_texts(word_data)

In [16]:
tok.word_index

{'新款': 1,
 '1': 2,
 '汽车': 3,
 '时尚': 4,
 '黑色': 5,
 '鞋': 6,
 '2016': 7,
 '休闲': 8,
 '款': 9,
 '专用': 10,
 '2': 11,
 '5': 12,
 '男士': 13,
 '跟': 14,
 '手机': 15,
 '适用': 16,
 '女': 17,
 '套装': 18,
 '男': 19,
 '韩版': 20,
 '夏季': 21,
 '儿童': 22,
 '3': 23,
 '包': 24,
 '于': 25,
 '透气': 26,
 '壳': 27,
 '套': 28,
 '休闲鞋': 29,
 '米': 30,
 '户外': 31,
 '男鞋': 32,
 '真皮': 33,
 '垫': 34,
 '红色': 35,
 '6': 36,
 '4': 37,
 '脚垫': 38,
 '女鞋': 39,
 '男女': 40,
 '白色': 41,
 '全': 42,
 '蓝色': 43,
 '创意': 44,
 '宝宝': 45,
 '防水': 46,
 '简约': 47,
 '四季': 48,
 '女士': 49,
 '8': 50,
 '保护套': 51,
 '凉鞋': 52,
 '牛皮': 53,
 '四件套': 54,
 '皮鞋': 55,
 '系列': 56,
 '专车': 57,
 '情侣': 58,
 '包围': 59,
 '灯': 60,
 '通用': 61,
 '纯棉': 62,
 '加厚': 63,
 '小': 64,
 '运动': 65,
 '单鞋': 66,
 '商务': 67,
 '学生': 68,
 '大': 69,
 '2015': 70,
 '送': 71,
 '潮流': 72,
 '客厅': 73,
 '婴儿': 74,
 '带': 75,
 '新': 76,
 '卡通': 77,
 '号': 78,
 '全棉': 79,
 '英伦': 80,
 '7': 81,
 '0': 82,
 '寸': 83,
 '可': 84,
 '现代': 85,
 '鞋子': 86,
 'led': 87,
 '单肩': 88,
 '玩具': 89,
 '黑': 90,
 '被': 91,
 '座垫': 92,
 '39': 93,
 '礼物': 94,

In [17]:
len(tok.word_index)

274420

In [18]:
x_train = tok.texts_to_sequences(word_data_train)

In [19]:
x_test = tok.texts_to_sequences(word_data_test)

In [20]:
x_train = sequence.pad_sequences(x_train, maxlen=20)
x_test  = sequence.pad_sequences(x_test, maxlen=20) 

In [21]:
x_test

array([[   155,   1487,    105, ...,      9,      2,    557],
       [     0,      0,      0, ..., 162550, 108802, 162551],
       [    91,    278,    149, ...,     11,     37,    548],
       ...,
       [     0,      0,      0, ...,    366,   7531, 152760],
       [     0,      0,      0, ...,    151,     47,     35],
       [     0,      0,      0, ...,  10689,     75,    565]])


## GRU Model

In [22]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import tensorflow.contrib.layers as layers
class GruModel(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(self, 
                 sequence_length, 
                 num_classes, 
                 embedding_size,  
                 num_filters, 
                 l2_reg_lambda=0.0, 
                 n_layer = 1, 
                 hidden_size = 32, 
                 batch_size = 256, 
                 vac_size = 27440):
        """
        sequence_length : 一个句子的长度（词的个数）
        embedding_size : 词向量的长度
        num_classes : 三个标签的类别数
        vac_size : 词的个数
        """
        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y1 = tf.placeholder(tf.float32, [None, num_classes[0]], name="input_y1")
        self.input_y2 = tf.placeholder(tf.float32, [None, num_classes[1]], name="input_y2")
        self.input_y3 = tf.placeholder(tf.float32, [None, num_classes[2]], name="input_y3")
        self.keep_prob = tf.placeholder(tf.float32, name="keep_prob")
        self.n_layer = n_layer
        self.batch_size = batch_size
        self.hidden_size = hidden_size

        
        with tf.name_scope("embedding"):
            W = tf.Variable(
                tf.random_uniform([274420, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)

        l2_loss = tf.constant(0.0)
        
        self.embedded_input = tf.layers.batch_normalization(self.embedded_chars, training=is_training)
        self.embedded_input = tf.expand_dims(self.embedded_input, -1)
        
        with tf.variable_scope("cnn1"):
            filter_shape = [3, 3, 1, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            self.conv_output = tf.nn.conv2d(
                    self.embedded_input,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="SAME",
                    name="conv")
            self.conv_output = tf.nn.dropout(self.conv_output, self.keep_prob)
            
        #gru模型
        with tf.variable_scope('inception_text'):
            self.output = self.text_inception(self.conv_output)
        
        with tf.variable_scope("cnn2"):
            filter_shape = [5, 5, 16, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            self.conv_output = tf.nn.conv2d(
                    self.output,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
            self.conv_output = tf.nn.dropout(self.conv_output, self.keep_prob)
        print(self.conv_output)
        self.h_drop = tf.reshape(self.conv_output,(-1, 16 * 20 * 4))
        print(self.h_drop.shape)
        num_filters_total = self.h_drop.shape[1]
        
        
        with tf.name_scope("output1"):
            W1 = tf.get_variable(
                "W11",
                shape=[num_filters_total, 64],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[64]), name="b11")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output1")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            self.output = tf.nn.dropout(self.output, self.keep_prob)
            W2 = tf.get_variable(
                "W12",
                shape=[64, num_classes[0]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[0]]), name="b12")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores1 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions1 = tf.argmax(self.scores1, 1, name="predictions")
            
        with tf.name_scope("output2"):
            W1 = tf.get_variable(
                "W21",
                shape=[num_filters_total, 128],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[128]), name="b21")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output2")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            self.output = tf.nn.dropout(self.output, self.keep_prob)
            W2 = tf.get_variable(
                "W22",
                shape=[128, num_classes[1]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[1]]), name="b22")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores2 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions2 = tf.argmax(self.scores2, 1, name="predictions")
            
        with tf.name_scope("output3"):
            W1 = tf.get_variable(
                "W31",
                shape=[num_filters_total, 256],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[256]), name="b31")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output3")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            self.output = tf.nn.dropout(self.output, self.keep_prob)
            W2 = tf.get_variable(
                "W32",
                shape=[256, num_classes[2]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[2]]), name="b32")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores3 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions3 = tf.argmax(self.scores3, 1, name="predictions")
        # Calculate mean cross-entropy loss
        
        with tf.name_scope("loss"):
            losses1 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores1, labels=self.input_y1)
            losses2 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores2, labels=self.input_y2)
            losses3 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores3, labels=self.input_y3)
            self.loss = 0.1 * tf.reduce_mean(losses1) + 0.2 * tf.reduce_mean(losses2) + 0.7*tf.reduce_mean(losses3) + l2_reg_lambda * l2_loss
            tf.summary.scalar('loss1',tf.reduce_mean(losses1))
            tf.summary.scalar('loss2',tf.reduce_mean(losses2))
            tf.summary.scalar('loss3',tf.reduce_mean(losses3))
            tf.summary.scalar('loss',self.loss)
        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions1 = tf.equal(self.predictions1, tf.argmax(self.input_y1, 1))
            correct_predictions2 = tf.equal(self.predictions2, tf.argmax(self.input_y2, 1))
            correct_predictions3 = tf.equal(self.predictions3, tf.argmax(self.input_y3, 1))
#           单独的准确率
            self.accuracy1 = tf.reduce_mean(tf.cast(correct_predictions1, "float"), name="accuracy1")
            self.accuracy2 = tf.reduce_mean(tf.cast(correct_predictions2, "float"), name="accuracy2")
            self.accuracy3 = tf.reduce_mean(tf.cast(correct_predictions3, "float"), name="accuracy3")
#           一起的准确率
            self.acc = tf.reduce_mean(tf.cast(correct_predictions1, "float")*tf.cast(correct_predictions2, "float")*tf.cast(correct_predictions3, "float"))
            tf.summary.scalar('acc1',self.accuracy1)
            tf.summary.scalar('acc2',self.accuracy2)
            tf.summary.scalar('acc3',self.accuracy3)
            tf.summary.scalar('acc',self.acc)
            
    def text_inception(self, inputs):
        output = []
        with tf.name_scope("conv-1"):
            filter_shape = [1, 1, num_filters, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            conv = tf.nn.conv2d(
                    inputs,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
            conv = tf.nn.dropout(conv, self.keep_prob)
            output.append(conv)
        with tf.name_scope("conv-2"):
            filter_shape1 = [1, 1, num_filters, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape1, stddev=0.1), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            conv1 = tf.nn.conv2d(
                    inputs,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
            filter_shape2 = [3, 3, num_filters, 2 * num_filters]
            W1 = tf.Variable(tf.truncated_normal(filter_shape2, stddev=0.1), name="W")
            b1 = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            conv2 = tf.nn.conv2d(
                    conv1,
                    W1,
                    strides=[1, 1, 1, 1],
                    padding="SAME",
                    name="conv")
            conv2 = tf.nn.dropout(conv2, self.keep_prob)
            output.append(conv2)
        with tf.name_scope("conv-4"):
            filter_shape = [3, 3, num_filters, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            conv = tf.nn.conv2d(
                    inputs,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="SAME",
                    name="conv")
            conv = tf.nn.dropout(conv, self.keep_prob)
            output.append(conv)
        print(tf.concat(output, 3).shape)
        return tf.concat(output, 3)

In [27]:

# batch 生成函数
import math
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
  
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = np.min(((batch_num + 1) * batch_size, data_size))
            yield shuffled_data[start_index:end_index]
            

## 参数

In [28]:
# 特征数这里是第三级标签的特征数
max_features = 1192

# 句子填充的长度
sequence_len = 20

# batch 大小
batch_size = 512 

# 迭代次数
epochs = 50

is_training = True

# 词向量长度
embedding_dims = 24

# gru  的filters
num_filters = 4

# filter 的大小
filter_size = [1, 2, 3, 4, 5]

# 三个标签的类数
num_classes = [22, 191, 1192]

In [29]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from tensorflow.contrib import learn



FLAGS = tf.flags.FLAGS

def train(x_train, y_train, x_dev, y_dev):
    # Training
    # ==================================================
    
    with tf.Graph().as_default():
        
        
        session_conf = tf.ConfigProto()
        sess = tf.Session(config=session_conf)
        
        with sess.as_default():
            cnn = GruModel(sequence_length = sequence_len, 
                          num_classes = num_classes, 
                          embedding_size = embedding_dims, 
                          num_filters = num_filters)
            saver=tf.train.Saver(max_to_keep=4)
            # Define Training procedure

            global_step = tf.Variable(0, name="global_step", trainable=False)
#           学习率衰减
            learning_rate = tf.train.exponential_decay(
                0.01,
                global_step,
                x_train.shape[0] / batch_size,
                0.99,
                staircase=True,)
            optimizer = tf.train.AdamOptimizer(learning_rate)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Initialize all variables
            sess.run(tf.global_variables_initializer())
            
            merged = tf.summary.merge_all()
            
            writer = tf.summary.FileWriter("fwwb",sess.graph)
            acc_sum = np.array([0.0,0.0,0.0,0.0])
            loss_sum = 0
            
            def train_step(x_batch, y_batch1, y_batch2, y_batch3, loss_sum, acc_sum, interal = 10):
  
                """
                A single training step
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y1: y_batch1,
                  cnn.input_y2: y_batch2,
                  cnn.input_y3: y_batch3,
                  cnn.keep_prob: 0.9
                }

#         三个标签单独的准确率， 一起的准确率共四个准确率
                rs, _, step,  loss, accuracy1, accuracy2, accuracy3, acc = sess.run(
                    [merged, train_op, global_step, cnn.loss, cnn.accuracy1, cnn.accuracy2, cnn.accuracy3, cnn.acc],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                acc_sum += [accuracy1, accuracy2, accuracy3, acc]
                loss_sum += loss
                if step % interal == 0:
                    saver.save(sess, "model/inception_model", global_step=step)
                    print("{}: step {}, loss {:g}, acc {}".format(time_str, step, loss_sum/interal, acc_sum/interal))
                    loss_sum = 0
                    acc_sum = np.array([0.0,0.0,0.0,0.0])
                    writer.add_summary(rs, step)
                return loss_sum, acc_sum
            # 评估步骤
            def dev_step(x_batch, y_batch):
                """
                Evaluates model on a dev set
                """

                dev_batches = batch_iter(
                    list(zip(x_batch, y_batch[0], y_batch[1], y_batch[2])), batch_size, 1)
                step = 0
                loss = 0
                accuracy = np.array([0.0, 0.0, 0.0,0.0])
                for batch in dev_batches:
                    x_batch_dev, y_batch1_dev, y_batch2_dev, y_batch3_dev = zip(*batch)
                    feed_dict = {
                      cnn.input_x: x_batch_dev,
                      cnn.input_y1: y_batch1_dev,
                      cnn.input_y2: y_batch2_dev,
                      cnn.input_y3: y_batch3_dev,
                      cnn.keep_prob: 1
                    }
                    if len(x_batch_dev) < batch_size:
                        continue
                    step,  temp_loss, temp_accuracy1, temp_accuracy2, temp_accuracy3 , acc= sess.run(
                        [global_step,cnn.loss, cnn.accuracy1, cnn.accuracy2, cnn.accuracy3, cnn.acc],
                        feed_dict)
                    accuracy[0] += temp_accuracy1 * len(x_batch_dev)
                    accuracy[1] += temp_accuracy2 * len(x_batch_dev)
                    accuracy[2] += temp_accuracy3 * len(x_batch_dev)
                    accuracy[3] += acc * len(x_batch_dev)
                    loss += temp_loss * len(x_batch_dev)
                accuracy /= x_batch.shape[0]
                loss /= x_batch.shape[0]
                time_str = datetime.datetime.now().isoformat()                    

                print("{}: step {}, loss {:g}, acc {} ".format(time_str, step, loss, accuracy))


            # Generate batches
            batches = batch_iter(
                list(zip(x_train, y_train[0],y_train[1], y_train[2] )), batch_size, epochs)
            # Training loop. For each batch...
            for batch in batches:
                x_batch, y_batch1, y_batch2, y_batch3 = zip(*batch)
                if len(x_batch) < batch_size:
                    continue
                loss_sum, acc_sum = train_step(x_batch, y_batch1, y_batch2, y_batch3,  loss_sum, acc_sum, 40)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % 200 == 0:
                    print("\nEvaluation:")
                    dev_step(x_dev, y_dev)
                    print("")

In [ ]:
y_train = [train_y_label1_one_hot,train_y_label2_one_hot,train_y_label3_one_hot]
y_test = [test_y_label1_one_hot,test_y_label2_one_hot,test_y_label3_one_hot]
train(x_train, y_train,  x_test, y_test)

(?, 20, 24, 16)
Tensor("cnn2/dropout/mul:0", shape=(?, 16, 20, 4), dtype=float32)
(?, 1280)
2019-03-03T11:45:07.427926: step 40, loss 3.88368, acc [0.42802734 0.35566406 0.25712891 0.19150391]
2019-03-03T11:45:11.918623: step 80, loss 2.54691, acc [0.63896484 0.56005859 0.47509766 0.37773438]
2019-03-03T11:45:16.657132: step 120, loss 1.99194, acc [0.73476562 0.65048828 0.57255859 0.48413086]
2019-03-03T11:45:21.488536: step 160, loss 1.69627, acc [0.78100586 0.69863281 0.62504883 0.53818359]
2019-03-03T11:45:26.477262: step 200, loss 1.5102, acc [0.80976563 0.725      0.65458984 0.57172852]

Evaluation:
2019-03-03T11:45:35.418647: step 200, loss 1.32893, acc [0.83475658 0.75455756 0.69211825 0.62106939] 

2019-03-03T11:45:39.652730: step 240, loss 1.35429, acc [0.82792969 0.75043945 0.68232422 0.60224609]
2019-03-03T11:45:44.238968: step 280, loss 1.28994, acc [0.84316406 0.76064453 0.69755859 0.62250977]
2019-03-03T11:45:48.632506: step 320, loss 1.20029, acc [0.84672852 0.77426758 0

2019-03-03T11:51:12.774769: step 2600, loss 0.287411, acc [0.96263853 0.94214578 0.9291013  0.89403238] 

2019-03-03T11:51:16.420319: step 2640, loss 0.218603, acc [0.96000977 0.94453125 0.93354492 0.88525391]
2019-03-03T11:51:20.589602: step 2680, loss 0.227332, acc [0.95864258 0.94077148 0.93183594 0.88217773]
2019-03-03T11:51:24.727869: step 2720, loss 0.228621, acc [0.95961914 0.94399414 0.93164062 0.88300781]
2019-03-03T11:51:28.850603: step 2760, loss 0.2238, acc [0.95991211 0.94414062 0.93095703 0.88334961]
2019-03-03T11:51:32.981490: step 2800, loss 0.242632, acc [0.95629883 0.93745117 0.92548828 0.87626953]

Evaluation:
2019-03-03T11:51:40.216222: step 2800, loss 0.283534, acc [0.96280872 0.94301675 0.92959185 0.89429266] 

2019-03-03T11:51:43.850871: step 2840, loss 0.234794, acc [0.95932617 0.94213867 0.92856445 0.87851563]
2019-03-03T11:51:48.016877: step 2880, loss 0.237248, acc [0.95546875 0.940625   0.92617187 0.87617188]
2019-03-03T11:51:52.149682: step 2920, loss 0.235

2019-03-03T11:57:03.601066: step 5200, loss 0.108239, acc [0.97348633 0.96767578 0.96533203 0.9284668 ]

Evaluation:
2019-03-03T11:57:11.002422: step 5200, loss 0.261295, acc [0.97465186 0.96229815 0.95015467 0.92843056] 

2019-03-03T11:57:14.609784: step 5240, loss 0.108462, acc [0.97641602 0.96821289 0.96518555 0.92919922]
2019-03-03T11:57:18.730174: step 5280, loss 0.11659, acc [0.97314453 0.96552734 0.96147461 0.9230957 ]
2019-03-03T11:57:22.850446: step 5320, loss 0.112098, acc [0.97524414 0.96572266 0.96191406 0.92548828]
2019-03-03T11:57:26.955761: step 5360, loss 0.118677, acc [0.97382813 0.96811523 0.96279297 0.92504883]
2019-03-03T11:57:31.147068: step 5400, loss 0.108798, acc [0.97397461 0.9675293  0.96445313 0.92763672]

Evaluation:
2019-03-03T11:57:38.509085: step 5400, loss 0.255201, acc [0.97470192 0.96264854 0.95263743 0.93055291] 

2019-03-03T11:57:42.165099: step 5440, loss 0.11786, acc [0.97285156 0.96230469 0.9628418  0.92280273]
2019-03-03T11:57:46.402264: step 548

2019-03-03T12:02:57.738900: step 7760, loss 0.0754239, acc [0.98149414 0.97578125 0.97548828 0.94555664]
2019-03-03T12:03:01.851905: step 7800, loss 0.0809063, acc [0.98012695 0.97348633 0.97363281 0.94262695]

Evaluation:
2019-03-03T12:03:09.082110: step 7800, loss 0.280451, acc [0.97798556 0.96650282 0.95607124 0.93857181] 

2019-03-03T12:03:12.836286: step 7840, loss 0.0454402, acc [0.98769531 0.98452148 0.98481445 0.96313477]
2019-03-03T12:03:16.984355: step 7880, loss 0.0462592, acc [0.98613281 0.98515625 0.9855957  0.96342773]
2019-03-03T12:03:21.185456: step 7920, loss 0.0480802, acc [0.98666992 0.98295898 0.98481445 0.96245117]
2019-03-03T12:03:25.380206: step 7960, loss 0.0521463, acc [0.98413086 0.98276367 0.9831543  0.95776367]
2019-03-03T12:03:29.488617: step 8000, loss 0.049319, acc [0.98583984 0.98374023 0.98476562 0.96245117]

Evaluation:
2019-03-03T12:03:36.742882: step 8000, loss 0.290258, acc [0.97782539 0.96734375 0.95724254 0.94028371] 

2019-03-03T12:03:40.352809: 

2019-03-03T12:08:51.347798: step 10320, loss 0.0338035, acc [0.98930664 0.98828125 0.98916016 0.97119141]
2019-03-03T12:08:55.456800: step 10360, loss 0.0361197, acc [0.9875     0.98681641 0.98891602 0.96879883]
2019-03-03T12:08:59.635312: step 10400, loss 0.0416031, acc [0.98701172 0.98676758 0.98632812 0.96591797]

Evaluation:
2019-03-03T12:09:06.958170: step 10400, loss 0.319137, acc [0.97908679 0.96921583 0.95902452 0.94403788] 

2019-03-03T12:09:10.574008: step 10440, loss 0.0417319, acc [0.98867187 0.98701172 0.98686523 0.96811523]
2019-03-03T12:09:14.706676: step 10480, loss 0.0408126, acc [0.98857422 0.98603516 0.98720703 0.96728516]
2019-03-03T12:09:18.835145: step 10520, loss 0.0437618, acc [0.98920898 0.98706055 0.98549805 0.9684082 ]
2019-03-03T12:09:23.046674: step 10560, loss 0.0456022, acc [0.98833008 0.98510742 0.98554688 0.96538086]
2019-03-03T12:09:27.161244: step 10600, loss 0.039812, acc [0.98740234 0.98510742 0.98740234 0.96591797]

Evaluation:
2019-03-03T12:09:34.

...

2019-03-03T12:54:54.451843: step 28040, loss 0.0200046, acc [0.9949707  0.99282227 0.9925293  0.98266602]
2019-03-03T12:54:58.614093: step 28080, loss 0.0174917, acc [0.99453125 0.99379883 0.99477539 0.98452148]
2019-03-03T12:55:02.865133: step 28120, loss 0.0132544, acc [0.99633789 0.99462891 0.99545898 0.98774414]
2019-03-03T12:55:07.013251: step 28160, loss 0.0136516, acc [0.99545898 0.99545898 0.99560547 0.98769531]
2019-03-03T12:55:11.117477: step 28200, loss 0.0107148, acc [0.99619141 0.99633789 0.99599609 0.9894043 ]

Evaluation:
2019-03-03T12:55:18.425888: step 28200, loss 0.456316, acc [0.98287099 0.97308012 0.9637798  0.95456957] 

2019-03-03T12:55:22.046625: step 28240, loss 0.0119336, acc [0.9956543  0.99536133 0.99638672 0.98867187]
2019-03-03T12:55:26.146204: step 28280, loss 0.0113722, acc [0.99560547 0.99594727 0.99667969 0.98916016]
2019-03-03T12:55:30.250602: step 28320, loss 0.0147625, acc [0.99487305 0.99550781 0.99526367 0.98710937]
2019-03-03T12:55:34.363012: step

2019-03-03T13:00:40.976955: step 30600, loss 0.0106021, acc [0.99521484 0.99584961 0.99677734 0.98886719]

Evaluation:
2019-03-03T13:00:48.218651: step 30600, loss 0.470044, acc [0.98318133 0.97319024 0.96354954 0.95476979] 

2019-03-03T13:00:51.838437: step 30640, loss 0.0117543, acc [0.99589844 0.99604492 0.99638672 0.98935547]
2019-03-03T13:00:55.947969: step 30680, loss 0.0128178, acc [0.99609375 0.99580078 0.99570313 0.98881836]
2019-03-03T13:01:00.132969: step 30720, loss 0.0131512, acc [0.99550781 0.99506836 0.99570313 0.9875    ]
2019-03-03T13:01:04.262269: step 30760, loss 0.0124798, acc [0.99609375 0.9956543  0.9956543  0.98876953]
2019-03-03T13:01:08.372421: step 30800, loss 0.0126981, acc [0.99521484 0.9956543  0.99648437 0.98833008]

Evaluation:
2019-03-03T13:01:15.685623: step 30800, loss 0.472093, acc [0.9832414  0.97316021 0.96394998 0.95519026] 

2019-03-03T13:01:19.304495: step 30840, loss 0.0141745, acc [0.99633789 0.99575195 0.99541016 0.98876953]
2019-03-03T13:01:2

2019-03-03T13:06:30.531584: step 33120, loss 0.0119705, acc [0.99599609 0.9949707  0.99648437 0.98862305]
2019-03-03T13:06:34.627225: step 33160, loss 0.0106881, acc [0.99628906 0.9956543  0.99614258 0.98916016]
2019-03-03T13:06:38.741424: step 33200, loss 0.0115274, acc [0.99594727 0.99589844 0.99589844 0.98867187]

Evaluation:
2019-03-03T13:06:46.016175: step 33200, loss 0.484526, acc [0.98343161 0.9732403  0.96407012 0.95556067] 

2019-03-03T13:06:49.631021: step 33240, loss 0.0117261, acc [0.99609375 0.99604492 0.99599609 0.98916016]
2019-03-03T13:06:53.750581: step 33280, loss 0.0120689, acc [0.99584961 0.99589844 0.99555664 0.98833008]
2019-03-03T13:06:57.970052: step 33320, loss 0.0129943, acc [0.99536133 0.99526367 0.99521484 0.98706055]
2019-03-03T13:07:02.149565: step 33360, loss 0.0116537, acc [0.99628906 0.99482422 0.99624023 0.98828125]
2019-03-03T13:07:06.324669: step 33400, loss 0.0133282, acc [0.99555664 0.99628906 0.99555664 0.98862305]

Evaluation:
2019-03-03T13:07:13

2019-03-03T13:12:28.338944: step 35640, loss 0.0118862, acc [0.99624023 0.99589844 0.99658203 0.98955078]
2019-03-03T13:12:32.445445: step 35680, loss 0.0120627, acc [0.99609375 0.99506836 0.99575195 0.98842773]
2019-03-03T13:12:36.568254: step 35720, loss 0.0118555, acc [0.99575195 0.99575195 0.99643555 0.98925781]
2019-03-03T13:12:40.735209: step 35760, loss 0.0120441, acc [0.99541016 0.99580078 0.99609375 0.98857422]
2019-03-03T13:12:44.914366: step 35800, loss 0.0128176, acc [0.99550781 0.99594727 0.99609375 0.98857422]

Evaluation:
2019-03-03T13:12:52.184515: step 35800, loss 0.493872, acc [0.98316131 0.97353062 0.96408013 0.95603119] 

2019-03-03T13:12:55.805762: step 35840, loss 0.0115536, acc [0.99575195 0.99584961 0.99658203 0.98930664]
2019-03-03T13:12:59.986646: step 35880, loss 0.0115576, acc [0.99609375 0.99472656 0.99580078 0.98833008]
2019-03-03T13:13:04.240300: step 35920, loss 0.010147, acc [0.99692383 0.99609375 0.99609375 0.99008789]
2019-03-03T13:13:08.342217: step 

2019-03-03T13:18:18.037515: step 38200, loss 0.011855, acc [0.99599609 0.99555664 0.99697266 0.98945313]

Evaluation:
2019-03-03T13:18:25.511987: step 38200, loss 0.506581, acc [0.98325141 0.97343051 0.96383986 0.95543053] 

2019-03-03T13:18:29.301427: step 38240, loss 0.0115476, acc [0.996875   0.99594727 0.99619141 0.99038086]
2019-03-03T13:18:33.485294: step 38280, loss 0.00903957, acc [0.99702148 0.99672852 0.99677734 0.99169922]
2019-03-03T13:18:37.667572: step 38320, loss 0.00952305, acc [0.99702148 0.99726563 0.99619141 0.99111328]
2019-03-03T13:18:41.766093: step 38360, loss 0.00892159, acc [0.99697266 0.9972168  0.99707031 0.99179688]
2019-03-03T13:18:45.952169: step 38400, loss 0.00961889, acc [0.996875   0.99677734 0.99658203 0.99082031]

Evaluation:
2019-03-03T13:18:53.164053: step 38400, loss 0.51202, acc [0.98354173 0.97377089 0.96371973 0.95571084] 

2019-03-03T13:18:56.780838: step 38440, loss 0.0118786, acc [0.99736328 0.99614258 0.99589844 0.99082031]


In [ ]:
test_y_label3_one_hot.shape

In [ ]:
train_y_label3_one_hot.shape